In [ ]:
# Install required libraries
!pip install transformers sentence-transformers pinecone-client[grpc] pandas


In this cell, the required Python libraries are installed. These include:
- `transformers` and `sentence-transformers` for natural language processing tasks.
- `pinecone-client[grpc]` for interacting with Pinecone's vector database.
- `pandas` for handling tabular data efficiently.

These libraries provide the foundation for building the RAG (Retrieval-Augmented Generation) model and processing financial data.


In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import os
from google.colab import files

This cell imports essential Python libraries:
- `pandas`: Used for creating and manipulating tabular data.
- `numpy`: Provides numerical computation capabilities, though not used directly in this script.
- `SentenceTransformer`: Used for creating text embeddings.
- `os` and `files`: Facilitate file handling and user uploads.

These imports prepare the environment for subsequent operations like embedding generation and file uploads.


In [ ]:
import pandas as pd

# Dummy P&L data for testing
pl_data = {
    "Quarter": [f"Q{(i % 4) + 1} {2000 + (i // 4)}" for i in range(100)],
    "Revenue": [
        150000 + (i * 30000) for i in range(100)
    ],
    "Operating Expenses": [
        80000 + (i * 10000) for i in range(100)
    ],
    "Net Income": [
        70000 + (i * 5000) for i in range(100)
    ],
    "Gross Profit": [
        100000 + (i * 10000) for i in range(100)
    ],
    "EBT (Earnings Before Tax)": [
        65000 + (i * 5000) for i in range(100)
    ]
}


# Find the maximum length of the lists in pl_data
max_length = max(len(value) for value in pl_data.values())

# Pad each list to the maximum length (if necessary)
for key, value in pl_data.items():
    while len(value) < max_length:
        value.append(None)

# Create the DataFrame
pl_df = pd.DataFrame(pl_data)




This cell generates dummy Profit and Loss (P&L) data for testing purposes:
- The dataset includes columns like `Quarter`, `Revenue`, `Operating Expenses`, `Net Income`, `Gross Profit`, and `EBT (Earnings Before Tax)`.
- It generates data for 100 quarters, spanning 25 years.
- Each column's values are calculated dynamically using mathematical expressions to simulate realistic financial data.

The data is then stored in a pandas DataFrame (`pl_df`), which will be used for further processing, such as embedding generation and querying.


In [ ]:
import time
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone client
pc = Pinecone(api_key="pcsk_4FW6nc_KCvYDUXeMCUT6Cb3YCfANmRnmyiibejoKybbGSnzmyRQUzNpsEEjpY6Qk1nPjJM")

# Define index name and configuration
index_name = "financial-qa-index"

# Check if the index exists, if not, create it
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # Matches SentenceTransformer model output
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print(f"Created index: {index_name}")
else:
    print(f"Index {index_name} already exists.")

# Connect to the index
index = pc.Index(index_name)
time.sleep(2)  # Allow time for index readiness

# View index stats
index_stats = index.describe_index_stats()
print("Index Stats:", index_stats)


Index financial-qa-index already exists.
Index Stats: {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 100}},
 'total_vector_count': 100}


This cell initializes and configures the Pinecone vector database:
- A Pinecone client is initialized using an API key.
- The index `financial-qa-index` is defined with a vector dimension of 384 (matching the SentenceTransformer output) and `cosine` similarity as the metric.
- If the index does not already exist, it is created on the AWS cloud in the `us-east-1` region.

Finally, the index connection is established, and its statistics are printed for verification.


In [ ]:
from sentence_transformers import SentenceTransformer

# Load the SentenceTransformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Add descriptions to the dataset
pl_df['Description'] = pl_df.apply(
    lambda row: f"Quarter: {row['Quarter']}, Revenue: {row['Revenue']}, "
                f"Operating Expenses: {row['Operating Expenses']}, "
                f"Net Income: {row['Net Income']}, Gross Profit: {row['Gross Profit']}, "
                f"EBT: {row['EBT (Earnings Before Tax)']}",
    axis=1
)

# Generate embeddings
embeddings = embedding_model.encode(pl_df['Description'].tolist(), show_progress_bar=True)
print("Generated Embeddings.")


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Generated Embeddings.


This cell loads a pre-trained SentenceTransformer model (`all-MiniLM-L6-v2`) for embedding generation:
- Each row of the P&L dataset is converted into a descriptive text format using the `apply` function.
- The resulting descriptions are passed to the SentenceTransformer model to generate embeddings, which are numerical vector representations of the text.
- These embeddings are essential for storing data in the Pinecone index and enabling efficient query retrieval.


In [ ]:
# Add embeddings to the index
vectors = [
    {"id": str(i), "values": embeddings[i], "metadata": {"text": pl_df['Description'][i]}}
    for i in range(len(embeddings))
]

index.upsert(vectors=vectors)
print(f"Inserted {len(vectors)} records into the index.")


Inserted 100 records into the index.


This cell is responsible for uploading the generated embeddings into the Pinecone vector database. The embeddings represent the financial data in a format suitable for similarity-based retrieval.

1. **Prepare Vectors for Upload**:
   - A list of dictionaries, `vectors`, is created where each dictionary represents a single embedding and its associated metadata.
   - Each dictionary contains:
     - **`id`**: A unique identifier for the vector (based on the row index of the financial data).
     - **`values`**: The embedding vector generated for the financial data description.
     - **`metadata`**: Additional context in the form of text descriptions of the corresponding financial data.

2. **Upload Vectors to Pinecone**:
   - The `index.upsert()` method uploads the list of vectors to the Pinecone index (`index`).
   - Pinecone uses the embeddings to build a vector database for efficient similarity-based retrieval during query processing.

3. **Output Confirmation**:
   - After uploading the vectors, the code prints the total number of vectors successfully inserted into the index.

**Purpose**:
- This step ensures that the processed financial data (in vector form) is stored in Pinecone, enabling the Retrieval-Augmented Generation (RAG) system to fetch relevant context for user queries in subsequent steps.


In [ ]:
from transformers import pipeline

# Initialize FLAN-T5 model for question answering
qa_pipeline = pipeline('text2text-generation', model='google/flan-t5-base')

# Function to retrieve relevant context
def retrieve_relevant_context(query, top_k=5):
    query_embedding = embedding_model.encode(query)  # Convert query to vector
    search_results = index.query(
        vector=query_embedding.tolist(),  # Ensure this is a list of floats
        top_k=top_k,  # Number of results to retrieve
        include_metadata=True  # Include metadata in the response
    )
    # Extract relevant context (text) from metadata
    return [result['metadata']['text'] for result in search_results['matches']]

# RAG QA function
def rag_qa(query):
    try:
        # Retrieve context from Pinecone
        context = retrieve_relevant_context(query)

        # Prepare input for the FLAN-T5 model
        input_text = f"Context: {' '.join(context)} Question: {query}"

        # Get the answer from FLAN-T5 model
        answer = qa_pipeline(input_text, max_length=100, truncation=True)

        # Return the generated answer
        return answer[0]['generated_text']

    except Exception as e:
        return f"Error: {str(e)}"

Device set to use cpu


This cell initializes the QA pipeline and defines the core logic for query-answer generation using a Retrieval-Augmented Generation (RAG) approach:

1. **FLAN-T5 Model Initialization**:
   - The `qa_pipeline` is created using the FLAN-T5 model (`google/flan-t5-base`), which is optimized for text-to-text generation tasks like question answering.

2. **Context Retrieval**:
   - The `retrieve_relevant_context` function queries the Pinecone index using a given user query.
   - The query is first converted into an embedding vector using the SentenceTransformer model.
   - The Pinecone index is searched for the `top_k` most relevant entries based on cosine similarity.
   - The function extracts and returns the relevant text (metadata) associated with the matching entries.

3. **Answer Generation**:
   - The `rag_qa` function combines the retrieved context with the user query to construct a detailed input prompt for the FLAN-T5 model.
   - The model generates an answer based on the input context and query, providing a concise response to the user's question.

4. **Error Handling**:
   - The `rag_qa` function includes error handling to gracefully report issues, such as query processing failures or model errors.

This cell demonstrates the integration of retrieval (Pinecone) and generation (FLAN-T5), forming the backbone of the QA system for answering financial queries.


In [ ]:
# Multiple queries test
queries = [
    "What is the gross profit for Q3 2024.?",
    "How do the net income and operating expances compare to Q1 2024",
    "What is the historical trend in revenue per customer between 2015 and 2024?",
    "How has the operating income changed from Q2 2015 to Q4 2020?",
    "What was the percentage change in net income from Q1 2012 to Q1 2023?",
    "Which quarter in 2019 had the highest gross profit, and how much was it?",
    "Compare the net income growth in 2016 and 2017, and determine which year had higher growth.",
    "How much did capital expenditures increase or decrease from Q2 2015 to Q2 2016?",
    "What was the average EPS for all of 2022, and how does it compare to the EPS for Q1 2022?",
    "What was the compound annual growth rate (CAGR) of revenue from Q1 2018 to Q1 2024?",
    "What is the percentage of operating expenses to revenue for Q3 2020?",
    "Which quarter had the highest operating expenses from 2000 to 2015, and what was the value?",
    "If the EPS for Q2 2021 had grown by 10%, what would the new EPS have been?",
    "Calculate the average revenue per quarter from Q1 2010 to Q4 2024 and compare it to the average revenue from 2005 to 2009.",
    "What is the ratio of capital expenditures to revenue for Q3 2020?",
    "In which quarter between 2010 and 2024 did net income grow by the highest percentage compared to the previous quarter?",
    "If the company maintained its Q2 2024 revenue growth rate for the next 5 years, what would the projected revenue be for Q2 2029?",
    "How much higher was the net income in Q4 2022 compared to Q4 2021, and what was the percentage increase?",
    "Was there a significant difference in capital expenditures between 2017 and 2018, and what caused it?",
    "Which quarter between 2005 and 2020 had the highest year-over-year EPS growth, and what was the exact growth percentage?",
    "Calculate the rolling 4-quarter average revenue for Q1 2015 to Q4 2024 and explain the trends over time.",
    "What is the cumulative net income for the years 2010 to 2015, and how does it compare to the cumulative revenue for the same period?",
    "What would the company’s net income have been in Q4 2010 if operating expenses were reduced by 10%?",
    "Compare the growth in revenue for the first half of 2022 (Q1 and Q2) to the first half of 2021, and provide both absolute and percentage growth.",
    "What is the ratio of operating income to operating expenses for Q3 2023?",
    "What was the average growth rate in EPS from 2015 to 2020?",
    "How much capital expenditure would have been needed in Q2 2023 to maintain the company’s growth rate from Q2 2022?",
    "What is the percentage change in net income from Q1 2021 to Q1 2023, adjusted for inflation?",
    "Which quarter between 2000 and 2010 had the highest gross profit margin, and what was the value?",
    "What is the correlation between revenue and capital expenditures from 2000 to 2024?",
    "How much would the net income need to increase each quarter in 2024 to achieve a 15% year-over-year growth for the full year?",
    "Which quarter had the highest year-over-year percentage increase in operating income between 2015 and 2020?",
    "What is the total net income for the years 2000 to 2004, and how does it compare to the total operating expenses for the same period?",
    "Compare the company’s EPS performance during the 2008 financial crisis (2008-2009) with the recovery period in 2010-2011.",
    "What would have been the capital expenditures in Q1 2021 if the company reduced its spending by 5% from the previous quarter?",
    "How did operating income change as a percentage of revenue from 2012 to 2017?",
    "Which quarter had the highest EPS growth from Q1 2020 to Q4 2023?",
    "What is the expected revenue growth in Q1 2025, based on the average growth rate in Q1 over the last 10 years?",
    "What is the ratio of net income to revenue for Q2 2014?",
    "How did the company’s operating income evolve from Q3 2000 to Q3 2010?",
    "Compare the revenue and net income growth rates from 2005 to 2010, and explain any notable differences.",
    "What is the total operating income for the years 2015 to 2020, and how does it compare to the total net income for the same period?",
    "Which quarter had the steepest drop in gross profit margin between 2000 and 2024, and what caused it?",
    "What is the variance of operating expenses for the years 2010 to 2020?",
    "How would the company's total revenue change if net income increased by 15% year-over-year from 2024 to 2029?",
    "What was the highest revenue achieved in a single quarter between 2000 and 2024, and how does it compare to the average revenue?",
    "Which quarter had the largest difference between operating income and net income, and what caused it?",
    "What is the trend in gross profit margin from 2010 to 2024, and what major events affected it?",
    "What was the average revenue per employee in 2023, assuming headcount data is available?",
    "What is the compound growth rate of capital expenditure between 2010 and 2023?",
    "What is the highest expense category contributing to operating expenses between 2015 and 2025?"
    "What was the highest year-over-year growth in gross profit between 2005 and 2024?",
    "What was the lowest EPS recorded between 2000 and 2024, and which quarter was it?",
    "What is the trend in operating margin from 2000 to 2024?",
    "Which quarter had the highest revenue-to-expense ratio between 2000 and 2024?",
    "What is the average quarterly growth rate in revenue over the past 5 years?",
    "How much has the dividend payout ratio fluctuated from 2015 to 2024?",
    "What percentage of total expenses was accounted for by marketing between 2015 and 2020?",
    "How much revenue did the company generate per employee in 2022?",
    "What is the historical trend of return on assets (ROA) between 2010 and 2024?",
    "What was the highest quarter-over-quarter revenue growth rate in the last 10 years?",
    "Compare the average gross profit in the first half of the year (H1) with the second half (H2) for 2020-2024.",
    "How did interest expenses impact net income between 2018 and 2023?",
    "Which year had the largest decline in operating margin between 2000 and 2024?",
    "What is the average annual dividend growth rate from 2010 to 2024?",
    "How did share buybacks affect the EPS between 2015 and 2024?",
    "Which quarter in the last decade had the highest return on equity (ROE)?",
    "What was the highest market capitalization achieved between 2000 and 2024?",
    "How has the company’s debt-to-equity ratio evolved from 2010 to 2024?",
    "What is the correlation between revenue and operating income from 2000 to 2024?",
    "What was the quarterly revenue trend in 2020 compared to 2021 during the pandemic?",
    "How much cash flow did the company generate in Q4 2023, and how does it compare to Q4 2022?",
    "What was the highest growth rate in cash flow from operations between 2015 and 2024?",
    "Which quarter had the steepest decline in revenue during the 2008 financial crisis?",
    "What is the historical average dividend yield between 2010 and 2024?",
    "How did foreign exchange (forex) impact revenue in 2023 compared to 2022?",
    "Which quarter between 2015 and 2024 had the highest free cash flow (FCF)?",
    "What is the average tax rate the company has paid between 2010 and 2024?",
    "How did restructuring costs affect net income during the 2020 pandemic?",
    "What is the compound annual growth rate (CAGR) of net income from 2000 to 2024?",
    "What was the highest quarterly gross margin achieved from 2010 to 2024?",
    "What is the percentage change in debt levels between 2015 and 2024?",
    "Which quarter had the highest increase in R&D expenses between 2010 and 2024?",
    "How much net income was retained as retained earnings between 2015 and 2024?",
    "How did acquisitions impact the revenue trend from 2018 to 2024?",
    "What is the average annual growth rate of cash flow from investing activities between 2015 and 2024?",
    "What was the impact of inflation on operating expenses from 2020 to 2024?",
    "What was the average interest coverage ratio between 2000 and 2024?",
    "What is the correlation between capital expenditures and revenue growth from 2000 to 2024?",
    "Which year between 2010 and 2020 had the highest net profit margin?",
    "What was the impact of tax rate changes on EPS from 2015 to 2024?",
    "How much cash did the company generate per share in 2022?",
    "What is the cumulative gross profit between 2015 and 2024?",
    "Which quarter had the highest net income margin between 2000 and 2024?",
    "What is the long-term trend in dividend payouts compared to net income?",
    "Which quarter between 2015 and 2020 had the highest increase in gross profit?",
    "What is the total revenue generated between 2000 and 2024?",
    "What is the average price-to-earnings (P/E) ratio for the last 10 years?",
    "What was the average gross margin percentage during the pandemic years (2020-2022)?",
    "What was the impact of capital expenditures on free cash flow from 2015 to 2024?",
    "How much revenue growth was driven by new product launches between 2018 and 2024?",
    "What was the impact of cost-cutting measures on operating income in 2023?",
    "How much net income was generated by the top-performing segment in 2022?",
    "Which product category contributed the most to revenue growth in 2023?",
    "What was the impact of share dilution on EPS between 2015 and 2024?",
    "How much revenue was generated from international markets between 2010 and 2024?",
    "What was the average EBITDA margin from 2015 to 2024?",
    "How much did the company’s total liabilities increase from 2010 to 2024?",
    "What is the rolling 12-month average revenue for the last 5 years?",
    "What was the impact of interest rate changes on net income between 2020 and 2024?",
    "How did changes in commodity prices affect the cost of goods sold (COGS) from 2015 to 2024?",
    "What was the largest percentage change in total assets between 2000 and 2024?",
    "What is the ratio of free cash flow to net income for the past 10 years?"
]


for query in queries:
    print(f"Query: {query}")
    print(f"Response: {rag_qa(query)}\n")

Query: What is the gross profit for Q3 2024.?
Response: 960000

Query: How do the net income and operating expances compare to Q1 2024
Response: Net income and operating expances are 205000 - 205000 = 5000.

Query: What is the historical trend in revenue per customer between 2015 and 2024?
Response: revenue per customer increased by 4%

Query: How has the operating income changed from Q2 2015 to Q4 2020?
Response: Revenue: 2040000

Query: What was the percentage change in net income from Q1 2012 to Q1 2023?
Response: .3

Query: Which quarter in 2019 had the highest gross profit, and how much was it?
Response: Q1

Query: Compare the net income growth in 2016 and 2017, and determine which year had higher growth.
Response: 2016

Query: How much did capital expenditures increase or decrease from Q2 2015 to Q2 2016?
Response: increase

Query: What was the average EPS for all of 2022, and how does it compare to the EPS for Q1 2022?
Response: 2700000

Query: What was the compound annual growt

KeyboardInterrupt: 

**Query Testing**:
   - Multiple financial queries are tested to demonstrate the pipeline's capability to handle complex financial questions.
   - Example questions include historical trends, percentage changes, and growth rates over specific periods.

The responses are printed for each query, showcasing the pipeline's functionality.